<a href="https://colab.research.google.com/github/AdamSe22/Gestion_des_dossiers_Medicaux/blob/main/TP_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chat Bot Gus

In [22]:
import re

def valider(prompt, reg, error_message):
    """ Demande les informations et valide le format selon un modèle regex. """
    while True:
        input = input(prompt)
        if re.match(reg, input):
            return input
        else:
            print(error_message)

def detect_cities(user_input):
    """ Détecte les villes dans le texte de l'utilisateur en utilisant des noms de villes courants. """
    cities = re.findall(r"\b(Paris|Nice|Casablanca|Rabat|St laurent du var)\b", user_input, re.IGNORECASE)
    if cities:
        return list(set(city.capitalize() for city in cities))
    else:
        return []

def main():
    print("GUS : Bienvenue sur GUS, le système de réservation de billets d'avion.")

    # Permet uniquement les lettres (y compris les lettres accentuées) sans espaces ni caractères spéciaux

    prenom = valider("GUS : Veuillez entrer votre prénom : ", r"^[A-Za-zÀ-ÖØ-öø-ÿ]+$", "Prénom invalide. Veuillez réessayer.")
    nom = valider("GUS : Veuillez entrer votre nom : ", r"^[A-Za-zÀ-ÖØ-öø-ÿ]+$", "Nom invalide. Veuillez réessayer.")
    # Vérifie un format standard d'adresse e-mail : texte avant et après le @, avec une extension de domaine d'au moins deux lettres
    email = valider("GUS : Veuillez entrer votre adresse e-mail : ", r"^[\w\.-]+@[\w\.-]+\.\w{2,}$", "E-mail invalide. Veuillez réessayer.")
    #Autorise un numéro de 9 à 15 chiffres, avec un + optionnel pour les codes internationaux.
    phone_number = valider("GUS : Veuillez entrer votre numéro de téléphone : ", r"^\+?[0-9]{9,15}$", "Numéro de téléphone invalide. Veuillez réessayer.")
    departure_date = valider("GUS : Veuillez entrer votre date de départ (jj/mm/aaaa) ou (jj-mm-aaa): ", r"^(0[1-9]|[12][0-9]|3[01])(/|-)(0[1-9]|1[0-2])(/|-)\d{4}$", "Date invalide. Veuillez réessayer.")
    return_date = valider("GUS : Veuillez entrer votre date de retour (jj/mm/aaaa) ou (jj-mm-aa): ", r"^(0[1-9]|[12][0-9]|3[01])(/|-)(0[1-9]|1[0-2])(/|-)\d{4}$", "Date invalide. Veuillez réessayer.")

    # Demande des villes à visiter
    print("GUS : Quelles villes souhaitez-vous visiter ?")
    user_response = input("Utilisateur : ")
    cities = detect_cities(user_response)

    if cities:
        print(f"GUS : Villes détectées où vous souhaitez aller : {', '.join(cities)}")
    else:
        print("GUS : Aucune ville de destination détectée.")

    # Confirmation de la réservation
    print(f"GUS : Merci {prenom} {nom} ! Votre réservation pour les dates du {departure_date} au {return_date} vers {', '.join(cities) if cities else 'aucune destination'} a bien été prise en compte.")
    print(f"GUS : Une confirmation sera envoyée à {email}.")


## TF-IDF

In [ ]:
!pip install -q nltk

In [8]:
# Importer les bibliothèques NLTK, scikit-learn, numpy, et math pour le traitement du texte et les calculs
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import math
# Télécharger les ressources NLTK nécessaires
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Charger le jeu de données 20 Newsgroups en limitant à 10% des documents pour réduire le temps de traitement
data = fetch_20newsgroups(subset='all')
documents = data.data[:int(0.1 * len(data.data))]  # Limitez à 10%
print(f"Nombre de documents chargés : {len(documents)}")


Nombre de documents chargés : 1884


In [6]:
# Afficher le nombre total de documents et le nombre de mots en utilisant split()
print(f"le nombre des documents : {len(documents)}")
total_words = sum(len(d.split()) for d in documents)
print(f"le nombre des mots : {total_words}")

# Afficher le nombre total de tokens en utilisant la tokenisation de NLTK
total_tokens = sum(len(word_tokenize(d)) for d in documents)
print(f"Nombre total de tokens : {total_tokens}")


Nombre total de documents : 1884
Nombre total de mots : 504273
Nombre total de tokens : 704924


In [12]:
#definir les Stop_words en anglais
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Initialiser le lemmatizer et le stemmer de NLTK pour la réduction des mots
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Créer une fonction pour tokeniser, lemmatiser, puis appliquer le stemming sur chaque mot
def preprocess_text(text):
    # Tokenisation du texte
    tokens = word_tokenize(text)
    # Suppression des stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Lemmatisation et stemming
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    # Reconstruction du texte prétraité
    return ' '.join(stemmed_tokens)

# Exemple de texte prétraité

print(preprocess_text(documents[0]))

: mamatha devineni ratnam < mr47+ @ andrew.cmu.edu > subject : pen fan reaction organ : post offic , carnegi mellon , pittsburgh , pa line : 12 nntp-posting-host : po4.andrew.cmu.edu sure basher pen fan pretti confus lack kind post recent pen massacr devil . actual , bit puzzl bit reliev . howev , go put end non-pittsburgh ' relief bit prais pen . man , kill devil wors thought . jagr show much better regular season stat . also lot fo fun watch playoff . bowman let jagr lot fun next coupl game sinc pen go beat pulp jersey anyway . disappoint see island lose final regular season game . pen rule ! ! !


In [14]:
# Appliquer la fonction de prétraitement sur chaque document
processed_documents = [preprocess_text(doc) for doc in documents]

In [15]:
# Calculer le nombre total de tokens après le prétraitement
processed_tokens = sum(len(word_tokenize(doc)) for doc in processed_documents)
print(f"Nombre total de tokens après prétraitement : {processed_tokens}")

# Calculer la taille du vocabulaire après prétraitement

vocabulaire = set(word for doc in processed_documents for word in doc.split())
print(f"Taille du vocabulaire après prétraitement : {len(vocabulaire)}")


Nombre total de tokens après prétraitement : 498366
Taille du vocabulaire après prétraitement : 44304


In [16]:
# Créer un dictionnaire associant chaque mot unique à un index
vocabulary_index = {word: i for i, word in enumerate(vocabulaire)}
print(vocabulary_index)


{'slowli': 0, 'fighter': 1, 'n-69f-2yd9w-02p': 2, "\\lqrs'-2pl+": 3, 'nicea': 4, 'start.r': 5, 'edm.93apr20145436': 6, 'rec/play': 7, 'warranti': 8, 'txd': 9, 'niel': 10, 'barton': 11, 'irna': 12, 'errat': 13, 'fasceti': 14, 'uss': 15, '1993apr13.132906.1827': 16, '20:13-14': 17, 'aldhfn.akron.oh.u': 18, '.556': 19, 'flamboy': 20, 'year-headstart': 21, 'promo/pictur': 22, 'bcarh10f': 23, 'mean': 24, '957': 25, 'mq': 26, 'manuev': 27, 'chinet.chinet.com': 28, 'trailer': 29, 'diskwrit': 30, 'husc8.harvard.edu': 31, 'risk': 32, '0.5': 33, 'moorhead': 34, 'unixg.ubc.ca': 35, 'proton/centaur': 36, '1945': 37, '1993mar20.050303.8401': 38, '88003': 39, 'trooper': 40, '18-to-19-hour': 41, 'watch': 42, 'athena': 43, 'latter': 44, 'mm': 45, '-jrl': 46, 'dass0001': 47, 'xu=+\\': 48, 'quantit': 49, 'pkgeragh': 50, '/-u': 51, 'intrepid': 52, 'qst7': 53, 'band': 54, 'x1y4': 55, 'c4acsl': 56, 'lamer': 57, 'improv': 58, 'gigo': 59, '0dh': 60, 'gvw': 61, 'pihati': 62, 'backrest': 63, '5s': 64, 'ca4gc0.

In [17]:
def calc_tf(doc):
    tokens = doc.split()
    total_tokens = len(tokens)
    tf = {}
    for word in tokens:
        tf[word] = tf.get(word, 0) + 1 / total_tokens
    return tf

# Exemple pour un document
print(calc_tf(processed_documents[0]))


{':': 0.04310344827586207, 'mamatha': 0.008620689655172414, 'devineni': 0.008620689655172414, 'ratnam': 0.008620689655172414, '<': 0.008620689655172414, 'mr47+': 0.008620689655172414, '@': 0.008620689655172414, 'andrew.cmu.edu': 0.008620689655172414, '>': 0.008620689655172414, 'subject': 0.008620689655172414, 'pen': 0.05172413793103449, 'fan': 0.017241379310344827, 'reaction': 0.008620689655172414, 'organ': 0.008620689655172414, 'post': 0.017241379310344827, 'offic': 0.008620689655172414, ',': 0.05172413793103449, 'carnegi': 0.008620689655172414, 'mellon': 0.008620689655172414, 'pittsburgh': 0.008620689655172414, 'pa': 0.008620689655172414, 'line': 0.008620689655172414, '12': 0.008620689655172414, 'nntp-posting-host': 0.008620689655172414, 'po4.andrew.cmu.edu': 0.008620689655172414, 'sure': 0.008620689655172414, 'basher': 0.008620689655172414, 'pretti': 0.008620689655172414, 'confus': 0.008620689655172414, 'lack': 0.008620689655172414, 'kind': 0.008620689655172414, 'recent': 0.00862068

In [18]:
N = len(processed_documents)
df = {}

# Comptez les fréquences des documents pour chaque mot
for doc in processed_documents:
    for word in set(doc.split()):
        df[word] = df.get(word, 0) + 1

# Calculez l'IDF avec lissage
idf = {word: math.log((N + 1) / (freq + 1)) + 1 for word, freq in df.items()}
print(idf)


{"'": 2.8246553984758895, 'better': 3.228477120840324, 'coupl': 4.18497427319252, 'pretti': 3.7376620551488546, 'organ': 1.0350913198112701, 'go': 2.443608817715871, 'non-pittsburgh': 7.848535919322166, 'rule': 4.2512236587337195, 'pen': 6.462241558202275, 'sure': 3.1123374709276703, 'game': 3.7880929087757464, 'offic': 4.110866301038797, 'devineni': 6.932245187448011, 'pa': 4.17223524741509, 'relief': 5.9026257702668525, 'massacr': 5.32280727501391, 'jersey': 6.2390980068880655, 'watch': 4.197877678028427, ',': 1.0339930220622076, 'show': 3.5244032630671867, '>': 1.4099845894151999, 'subject': 1.0, 'lack': 4.57139118632999, 'lot': 3.4058846628318493, 'end': 3.5858560422808505, 'see': 2.7305421069054105, 'much': 2.83790062522591, ':': 1.0, 'fo': 7.443070811214001, 'andrew.cmu.edu': 4.735020610111791, 'ratnam': 6.932245187448011, 'nntp-posting-host': 1.8683851321144571, 'pulp': 7.848535919322166, 'mamatha': 6.932245187448011, 'wors': 4.67048208897422, 'final': 3.946563249747521, 'basher

In [19]:
def calc_tf_idf(tf, idf):
    tf_idf = {}
    for word, tf_value in tf.items():
        tf_idf[word] = tf_value * idf.get(word, 0)
    return tf_idf

# Appliquez le calcul TF-IDF pour chaque document
tf_idf_d = [calc_tf_idf(calc_tf_idf(d), idf) for d in processed_documents]
print(tf_idf_d[0])  # Affiche le TF-IDF pour un document


{':': 0.04310344827586207, 'mamatha': 0.059760734374551816, 'devineni': 0.059760734374551816, 'ratnam': 0.059760734374551816, '<': 0.014286176274476968, 'mr47+': 0.059760734374551816, '@': 0.008625264178843127, 'andrew.cmu.edu': 0.040819143190618884, '>': 0.012155039563924136, 'subject': 0.008620689655172414, 'pen': 0.3342538737001177, 'fan': 0.07849481126410063, 'reaction': 0.046238183358053145, 'organ': 0.008923201032855778, 'post': 0.04841968660317871, 'offic': 0.03543850259516204, ',': 0.05348239769287281, 'carnegi': 0.04298564688269567, 'mellon': 0.042742794495051736, 'pittsburgh': 0.038781308896961735, 'pa': 0.03596754523633698, 'line': 0.008639002332595526, '12': 0.03327199890308527, 'nntp-posting-host': 0.016106768380297044, 'po4.andrew.cmu.edu': 0.05818899681598324, 'sure': 0.02683049543903164, 'basher': 0.06168438567898466, 'pretti': 0.032221224613352195, 'confus': 0.04375816549208952, 'lack': 0.03940854470974129, 'kind': 0.031158874264869883, 'recent': 0.0347484792531488, 'm

In [20]:
import pandas as pd

# Créez un DataFrame avec les mots en colonnes
data_TF_IDF = pd.DataFrame(tf_idf_d).fillna(0)
print(data_TF_IDF)


             :   mamatha  devineni    ratnam         <     mr47+         @  \
0     0.043103  0.059761  0.059761  0.059761  0.014286  0.059761  0.008625   
1     0.055172  0.000000  0.000000  0.000000  0.011429  0.000000  0.013800   
2     0.016639  0.000000  0.000000  0.000000  0.005515  0.000000  0.003330   
3     0.018987  0.000000  0.000000  0.000000  0.006992  0.000000  0.016887   
4     0.043478  0.000000  0.000000  0.000000  0.028821  0.000000  0.008700   
...        ...       ...       ...       ...       ...       ...       ...   
1879  0.111111  0.000000  0.000000  0.000000  0.000000  0.000000  0.027793   
1880  0.030702  0.000000  0.000000  0.000000  0.058147  0.000000  0.008777   
1881  0.086207  0.000000  0.000000  0.000000  0.042859  0.000000  0.025876   
1882  0.157895  0.000000  0.000000  0.000000  0.000000  0.000000  0.035106   
1883  0.012469  0.000000  0.000000  0.000000  0.000000  0.000000  0.002495   

      andrew.cmu.edu         >   subject  ...  bible-lov  skate

##  Reconnaissance d'Entités Nommées (NER) avec Huggingface


In [32]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load model directly
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [33]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'I-PER', 'score': 0.99914825, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'I-LOC', 'score': 0.9983669, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [34]:
from datasets import load_dataset

# Charger le jeu de données CoNLL-2003
dataset = load_dataset("conll2003")

# Afficher les divisions du jeu de données pour comprendre sa structure
print(dataset)


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] true
The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
